# Using the pluto MutualFundsIndia data-set

In [1]:
import pandas as pd
from sqlalchemy import func, or_, and_
import plutoPy
from plutoPy.model import RoutingSession, MutualFundsIndia

### who are the biggest 5 mutual fund managers?

In [2]:
lastAumDate = RoutingSession.session.query(func.max(MutualFundsIndia.AumFundwise.PERIOD)).scalar()
print(f"{lastAumDate}")

aums = (RoutingSession.session.query(MutualFundsIndia.AumFundwise.FUND, MutualFundsIndia.AumFundwise.AVG_AUM_WO_FOFD + MutualFundsIndia.AumFundwise.AVG_AUM_FOFD)
        .filter(MutualFundsIndia.AumFundwise.PERIOD == lastAumDate)
        .order_by((MutualFundsIndia.AumFundwise.AVG_AUM_WO_FOFD + MutualFundsIndia.AumFundwise.AVG_AUM_FOFD).desc())).all()

aumDf = pd.DataFrame(aums, columns=['FUND', 'AUM']) #analyze this!
print(aumDf[0:5])

2019-07-02 16:24:25,746 INFO sqlalchemy.engine.base.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2019-07-02 16:24:25,746 INFO sqlalchemy.engine.base.Engine ()
2019-07-02 16:24:25,749 INFO sqlalchemy.engine.base.Engine SELECT schema_name()
2019-07-02 16:24:25,750 INFO sqlalchemy.engine.base.Engine ()
2019-07-02 16:24:25,766 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-02 16:24:25,767 INFO sqlalchemy.engine.base.Engine ()
2019-07-02 16:24:25,770 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS NVARCHAR(60)) AS anon_1
2019-07-02 16:24:25,770 INFO sqlalchemy.engine.base.Engine ()
2019-07-02 16:24:25,775 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-07-02 16:24:25,776 INFO sqlalchemy.engine.base.Engine SELECT max([MF_FUNDWISE_AUM].[PERIOD]) AS max_1 
FROM [MF_FUNDWISE_AUM]
2019-07-02 16:24:25,776 INFO sqlalchemy.engine.base.Engine ()
2019-06-01
2019-07-02 16:24:25,780 INFO sqlalche

/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)
